In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge,Lasso,RidgeCV,LassoCV,ElasticNet,ElasticNetCV,LinearRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
import statsmodels.api as sm

In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/divyansha1115/Graduate-Admission-Prediction/master/Admission_Predict.csv")

In [3]:
from pandas_profiling import ProfileReport


ModuleNotFoundError: No module named 'pandas_profiling'

In [16]:
df.drop(columns=['Serial No.'],inplace=True)

In [17]:
y=df['Chance of Admit ']

In [20]:
x=df.drop(columns=['Chance of Admit '])

In [21]:
scaler= StandardScaler()

In [22]:
arr=scaler.fit_transform(x)

In [23]:
df1=pd.DataFrame(arr)

In [24]:
df1.profile_Report()

AttributeError: 'DataFrame' object has no attribute 'profile_Report'

In [25]:
df1.describe()

,0,1,2,3,4,5,6
count,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02
mean,-3.785861e-16,5.412337e-16,7.147061e-16,1.859624e-16,-3.019807e-16,8.076873e-16,2.942091e-17
std,1.001252e+00,1.001252e+00,1.001252e+00,1.001252e+00,1.001252e+00,1.001252e+00,1.001252e+00
min,-2.339367e+00,-2.542098e+00,-1.827457e+00,-2.386613e+00,-2.733036e+00,-3.020504e+00,-1.099975e+00
25%,-7.685900e-01,-7.274920e-01,-9.520286e-01,-8.949798e-01,-5.042604e-01,-7.201909e-01,-1.099975e+00
50%,1.679859e-02,-6.763531e-02,-7.660001e-02,9.944220e-02,5.293342e-02,1.859559e-02,9.091117e-01
75%,7.149218e-01,7.571856e-01,7.988286e-01,5.966532e-01,6.101273e-01,7.783704e-01,9.091117e-01
max,2.023903e+00,2.076899e+00,1.674257e+00,1.591075e+00,1.724515e+00,2.218165e+00,9.091117e-01


In [26]:
#multicoliarity
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_df=pd.DataFrame()

In [27]:
 vif_df['vif']=[variance_inflation_factor (arr,i) for i in range(arr.shape[1])]
 vif_df['feature']=x.columns

In [28]:
vif_df

,vif,feature
0,4.615516,GRE Score
1,4.288959,TOEFL Score
2,2.919606,University Rating
3,3.075504,SOP
4,2.431258,LOR
5,5.207403,CGPA
6,1.543312,Research


In [66]:
x_train,x_test,y_train,y_test=train_test_split(arr,y,test_size=0.25,random_state=90)

In [67]:
x_train

array([[-0.76859003,  0.09732886, -0.07660001, ...,  0.05293342,
        -0.63623792, -1.09997489],
       [ 0.10406399, -0.23259949, -0.07660001, ..., -0.50426044,
         0.085758  , -1.09997489],
       [ 1.41304503,  1.91193482,  1.67425725, ...,  1.16732114,
         1.98309612,  0.90911166],
       ...,
       [-0.50679383, -0.39756367, -0.07660001, ..., -0.50426044,
        -0.25005405,  0.90911166],
       [-0.68132463, -0.56252785, -0.95202863, ..., -1.06145431,
        -0.5690755 , -1.09997489],
       [-1.55397866, -2.21216962, -1.82745726, ..., -2.73303589,
        -2.11381096, -1.09997489]])

In [68]:
lr=LinearRegression()

In [69]:
lr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [70]:
pickle.dump(lr,open('admission_lr_model.pickle','wb'))

In [71]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [72]:
lr.predict([[337,118,0,4.5,4.5,9.65,1]])

array([10.70946183])

In [73]:
test1=scaler.transform([[337,118,0,4.5,4.5,9.65,1]])

In [74]:
test1

array([[ 1.76210664,  1.74697064, -2.70288589,  1.09386422,  1.16732114,
         1.76481828,  0.90911166]])

In [75]:
lr.predict([[ 1.76210664,  1.74697064, -2.70288589,  1.09386422,  1.16732114,
         1.76481828,  0.90911166]])

array([0.93819604])

In [76]:
model=pickle.load(open('admission_lr_model.pickle','rb'))

In [77]:
model.predict(test1)

array([0.93819604])

In [78]:
lr.score(x_test,y_test)#R2

0.8195438498415173

In [79]:
def adj_r2(x,y):
    r2=lr.score(x,y)
    n=x.shape[0]
    p=x.shape[1]
    adjusted_r2=1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [80]:
adj_r2(x_test,y_test)

0.8058134905903284

In [81]:
lr.coef_

array([0.02368302, 0.01022828, 0.0030594 , 0.0029901 , 0.02074152,
       0.07027647, 0.0113466 ])

In [82]:
lr.intercept_

0.7250414346277145

In [107]:
lassocv=LassoCV(alphas=None,cv=50,max_iter=2000000,normalize=True)#cv is technique to find out best param to find out to find model
lassocv.fit(x_train,y_train)

LassoCV(alphas=None, copy_X=True, cv=50, eps=0.001, fit_intercept=True,
        max_iter=2000000, n_alphas=100, n_jobs=None, normalize=True,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [108]:
lassocv.alpha_#lambda

7.142482788425483e-06

In [109]:
lasso=Lasso(alpha=lassocv.alpha_)
lasso.fit(x_train,y_train)

Lasso(alpha=7.142482788425483e-06, copy_X=True, fit_intercept=True,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [110]:
lasso.score(x_test,y_test)

0.8195389103189991

In [117]:
ridgecv=RidgeCV(alphas=np.random.uniform(0,10,50),cv=50,normalize=True)
ridgecv.fit(x_train,y_train)

RidgeCV(alphas=array([5.10154426, 6.09949493, 5.03366734, 6.43315412, 4.3526854 ,
       1.37404459, 7.01977547, 3.4419385 , 8.35102798, 3.03728537,
       0.22524308, 9.05936364, 0.86732842, 0.75938054, 4.49423656,
       9.41290395, 8.20410314, 5.54706016, 9.90951817, 2.11881389,
       3.93781   , 1.53137273, 2.36975305, 8.56889001, 7.80700877,
       5.58916016, 8.187896  , 2.03955869, 4.03356844, 0.31489501,
       7.07603706, 4.38308667, 9.01786278, 0.82921147, 5.26283417,
       1.15742078, 5.3365934 , 3.46911211, 7.92724728, 0.77859105,
       9.44579662, 6.32192929, 4.55447399, 1.44680581, 8.0958165 ,
       0.59565091, 7.84040161, 3.77228354, 6.52533387, 9.0516443 ]),
        cv=50, fit_intercept=True, gcv_mode=None, normalize=True, scoring=None,
        store_cv_values=False)

In [118]:
ridgecv.alpha_#alpha hyper param

0.22524307839711377

In [122]:
ridge_lr=Ridge(alpha=ridgecv.alpha_)
ridge_lr.fit(x_train,y_train)

Ridge(alpha=0.22524307839711377, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [123]:
ridge_lr.score(x_test,y_test)

0.819544771733923

In [125]:
elastic=ElasticNetCV(alphas=None,cv=10)
elastic.fit(x_train,y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=1000, n_alphas=100, n_jobs=None,
             normalize=False, positive=False, precompute='auto',
             random_state=None, selection='cyclic', tol=0.0001, verbose=0)

In [126]:
elastic.alpha_

0.00027116921785873786

In [127]:
elastic.l1_ratio_

0.5

In [130]:
elastic_lr=ElasticNet(alpha=elastic.alpha_,l1_ratio=elastic.l1_ratio)
elastic_lr.fit(x_train,y_train)

ElasticNet(alpha=0.00027116921785873786, copy_X=True, fit_intercept=True,
           l1_ratio=0.5, max_iter=1000, normalize=False, positive=False,
           precompute=False, random_state=None, selection='cyclic', tol=0.0001,
           warm_start=False)

In [131]:
elastic_lr.score(x_test,y_test)

0.8194560128084636